In [50]:
import requests
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from selenium import webdriver
import time
import random
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


chromedriver = os.environ['CHROME_DRIVER']
driver = webdriver.Chrome(chromedriver)

link = 'https://www3.wipo.int/wipogreen-database/'
driver.get(link)

In [51]:
tile_click = True
count = 1
cause_dictionary = {}

In [52]:
#clicks all buttons

#clicks all buttons

while tile_click:
    try:
        button = driver.find_element_by_xpath('//*[@id="wipo-int"]/div[1]/div/div[1]/div[3]/div/a[{}]'.format(count))
        time.sleep(5)
        category_name = driver.find_element_by_xpath('//*[@id="wipo-int"]/div[1]/div/div[1]/div[3]/div/a[{}]'.format(count)).text
        driver.execute_script("arguments[0].click();", button)
        count += 1
        #then show more
        #clicks all titles
        project_click = True
        project_count = 1
        while project_click:
            first_set = 1
            table = 1
            try:
                button = driver.find_element_by_xpath('//*[@id="wipo-int"]/div[2]/div/div[1]/div[2]/div[3]/div[2]/div[{}]/h4/a'.format(project_count))
                time.sleep(5)
                entry = button.text
                cause_dictionary[entry] = {} #this is our title
                cause_dictionary[entry]["category"] = category_name
                cause_dictionary[entry]["submitted_by"] = driver.find_element_by_xpath('//*[@id="wipo-int"]/div[2]/div/div[1]/div[2]/div[3]/div[2]/div[{}]/ul/li[2]/a'.format(project_count)).text
                driver.execute_script("arguments[0].click();", button)
                #code to scrape page
                cause_dictionary[entry]["published"] = driver.find_element_by_xpath('//*[@id="searchDetail"]/div/ul/li[2]/time').text
                cause_dictionary[entry]["updated"] = driver.find_element_by_xpath('//*[@id="searchDetail"]/div/ul/li[3]/time').text
                if driver.find_element_by_xpath('//*[@id="wipo-int"]/div[1]/div/div[1]/section/div/div[2]/h2[1]').text == 'Technology features':
                    cause_dictionary[entry]["proposal/needed"] = "proposal"
                else:
                    cause_dictionary[entry]["proposal/needed"] = "needed"
                scraper = True
                while scraper:
                    try:
                        feature_ = driver.find_element_by_xpath('//*[@id="wipo-int"]/div[1]/div/div[1]/section/div/div[2]/table/tbody/tr[{}]/th'.format(first_set))
                        try:
                            feature_text = driver.find_element_by_xpath('//*[@id="wipo-int"]/div[1]/div/div[1]/section/div/div[2]/table/tbody/tr[{}]/td/pre'.format(first_set))
                            first_set += 1
                            cause_dictionary[entry][feature_.text] = feature_text.text
                        except:
                            feature_text = driver.find_element_by_xpath('//*[@id="wipo-int"]/div[1]/div/div[1]/section/div/div[2]/table/tbody/tr[{}]/td'.format(first_set))
                            first_set += 1
                            cause_dictionary[entry][feature_.text] = feature_text.text
                    except:
                        try:
                            for table in range(1,3):
                                table_stop = True
                                first_set = 1
                                while table_stop:
                                    try:
                                        feature_ = driver.find_element_by_xpath('//*[@id="wipo-int"]/div[1]/div/div[1]/section/div/div[2]/table[{}]/tbody/tr[{}]/th'.format(table, first_set))
                                        feature_text = driver.find_element_by_xpath('//*[@id="wipo-int"]/div[1]/div/div[1]/section/div/div[2]/table[{}]/tbody/tr[{}]/td/pre'.format(table, first_set))
                                        first_set += 1
                                        cause_dictionary[entry][feature_.text] = feature_text.text
                                    except:
                                        try:
                                            feature_ = driver.find_element_by_xpath('//*[@id="wipo-int"]/div[1]/div/div[1]/section/div/div[2]/table[{}]/tbody/tr[{}]/th'.format(table, first_set))
                                            feature_text = driver.find_element_by_xpath('//*[@id="wipo-int"]/div[1]/div/div[1]/section/div/div[2]/table[{}]/tbody/tr[{}]/td'.format(table, first_set))
                                            first_set += 1
                                            cause_dictionary[entry][feature_.text] = feature_text.text
                                        except:
                                            first_set = 1
                                            table_stop = False
                            scraper = False
                        except:
                            scraper = False
                project_count += 1
                #return to page
                driver.execute_script("window.history.go(-1)")
            except:
                #then click next page
                try:
                    button = driver.find_element_by_xpath('//*[@title="Next"]')
                    time.sleep(5)
                    driver.execute_script("arguments[0].click();", button)
                    project_count = 1
                except:
                    #return to tiles
#                     df = pd.DataFrame.from_dict(cause_dictionary, orient='index')
#                     df.to_csv('{}_data.csv'.format(category_name))
                    project_click = False
                    link = 'https://www3.wipo.int/wipogreen-database/'
                    driver.get(link)
    except:
        tile_click = False

In [87]:
import pandas as pd
X = pd.DataFrame.from_dict(cause_dictionary, orient='index')

In [88]:
X.reset_index(inplace=True)

In [90]:
wipo_df = X.rename(columns={'index':'title', 'category':'sector', 'published':'date_posted', 'proposal/needed':'meta_category',\
                            'updated': 'last_updated', 'Benefits':'benefits', 'Resources to be saved or improved':'resources_saved',\
                            'Technical fields':'technical_fields', 'Stage of development':'development_stage',\
                            'Project summary':'project_summary', 'Technology type':'type', 'meta_category':'seeking_support'})

In [91]:
wipo_df['meta_org'] = 'wipo_green'

In [94]:
wipo_df['meta_category'] = wipo_df['meta_category'].apply(lambda x: 'need' if x is 'needed' else 'offer')

In [96]:
wipo_df.to_json('wipo_green.json', orient='records')